In [ ]:
import os, sys, pandas as pd, json, torch, torch.nn as nn, copy
sys.path.append("/content")
from src.utils import plot_obs_predict, fit_mlp_full, fit_mlp_full_nolog
os.chdir("/content")
with open("/content/credentials") as f:
    env_vars = f.read().split("\n")

for var in env_vars:
    key, value = var.split(" = ")
    os.environ[key] = value

storage_options = {"account_name":os.environ["ACCOUNT_NAME"],
                   "account_key":os.environ["BLOB_KEY"]}


In [ ]:

results = pd.read_csv(f"az://model-output/mlp/grid_search_metadata_500m_cloudthr80_lulcmndwi_masking_5folds_seed123.csv", storage_options=storage_options)
no_azimuth = ["mean_viewing_azimuth" not in results["features"][i] for i in range(0, (results.shape[0]))]
has_is_brazil = ["is_brazil" in results["features"][i] for i in range(0, (results.shape[0]))]
results_sub = copy.copy(results.loc[no_azimuth, :])
results_sub2 = copy.copy(results.loc[has_is_brazil, :])
results_to_use = results_sub2
results_to_use["mean_mse"] = (results_to_use["val_site_mse"] + results_to_use["val_pooled_mse"]) / 2

with open(results_to_use.iloc[results_to_use["mean_mse"].argmin(), :]["path"], "rb") as f:
    top_model = json.load(f)

print(top_model["features"])
print(top_model["learning_rate"])
print(top_model["features"])
print(top_model["layer_out_neurons"])
print(top_model["activation"])
print(top_model["weight_decay"])

In [ ]:
## Fit the full model based on top model above
from torch import nn

model_scale = "log"

if model_scale == "native":
    model_fun = fit_mlp_full_nolog
else:
    model_fun = fit_mlp_full

model_out = model_fun(
    features=top_model["features"],
    learning_rate=top_model["learning_rate"],
    batch_size=top_model["batch_size"],
    epochs=5000,
    storage_options=storage_options,
    activation_function=eval(f'nn.{top_model["activation"]}'),
    day_tolerance=top_model["day_tolerance"],
    cloud_thr=top_model["cloud_thr"],
    mask_method1="lulc",
    mask_method2="mndwi",
    min_water_pixels=20,
    layer_out_neurons=top_model["layer_out_neurons"],
    verbose=True,
    weight_decay=1e-2*0
)

In [ ]:
from sklearn.metrics import r2_score
import numpy as np
partition = "test"
if partition == "test":
    pred = pd.DataFrame(
        {
            "obs": model_out["y_obs_test"],
            "pred": model_out["y_pred_test"],
            "sample_id": model_out["y_test_sample_id"]
        }
    )
else:
    pred = pd.DataFrame(
        {
            "obs": model_out["y_obs_train"],
            "pred": model_out["y_pred_train"],
            "sample_id": model_out["y_train_sample_id"]
        }
    )
    print(f'R2 train: {r2_score(model_out["y_pred_train"], model_out["y_obs_train"])}')

pred_itv = pred.loc[["ITV" in x for x in pred["sample_id"]], :]
pred_data = pred

if model_scale == "native":
    ssc_pred = [x * 100 for x in pred_data["pred"]] #[(np.exp(x)) for x in pred_data["pred"]]
    ssc_obs = [x * 100 for x in pred_data["obs"]]
else: # [np.exp(x) for x in pred_data["obs"]]
    print(f'R2 (log scc): {r2_score(pred_data["pred"], pred_data["obs"])}')
    ssc_pred = [np.exp(x) for x in pred_data["pred"]]
    ssc_obs = [np.exp(x) for x in pred_data["obs"]]

log_ssc_obs = pred_data["obs"]

print(f'R2 (scc): {r2_score(ssc_pred, ssc_obs)}')
print(f'R2 (scc 0-100): {r2_score([ssc_pred[i] for i in range(0,len(ssc_obs)) if ssc_obs[i]<100], [x for x in ssc_obs if x<100])}')

# plot_obs_predict(pd.DataFrame({"pred": [ssc_pred[i] for i in range(0,len(ssc_obs)) if ssc_obs[i]<100], "obs":  [x for x in ssc_obs if x<100]}), f"Observed vs. Predicted, Test Partition")
plot_obs_predict(pd.DataFrame({"pred": ssc_pred, "obs": ssc_obs}), f"Observed vs. Predicted, Test Partition")
# plot_obs_predict(pd.DataFrame({"pred": pred_data["pred"], "obs":  pred_data["obs"]}), f"Observed vs. Predicted, Test Partition")
#plot_obs_predict(pd.DataFrame({"pred": model_out["y_pred_train"], "obs": model_out["y_obs_train"]}), f"Observed vs. Predicted, Training Partition")


In [ ]:
import fsspec
fs = fsspec.filesystem("az", **storage_options)

In [ ]:
fs.put_file("/content/output/mlp/top_model.pt", "model-output/mlp/top_model.pt", overwrite=True)
fs.put_file("/content/output/mlp/grid_search_metadata_v2.csv", "model-output/mlp/grid_search_metadata_v2.csv", overwrite=True)
fs.put_file("/content/output/mlp/top_model_metadata.pickle", "model-output/mlp/top_model_metadata.pickle", overwrite=True)

In [ ]:
paths_full = os.listdir("/content/output/mlp/500m_cloudthr80_lulcmndwi_masking_tmp_5fold")
paths = [os.path.basename(x) for x in paths_full]

for path in paths:
    fs.put_file(
        f"output/mlp/500m_cloudthr80_lulcmndwi_masking_tmp_5fold/{path}",
        f"model-output/mlp/500m_cloudthr80_lulcmndwi_masking_tmp_5fold/{path}"
    )

In [ ]:
from matplotlib import pyplot as plt

lost_list = model_out["train_loss"]

plt.figure(figsize=(8,6))
plt.plot(lost_list, color="teal", label="training")
plt.title(f"Loss by Epoch for Training Partition")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.ylim((0, 3))
plt.legend()
plt.show()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(8,8))
plt.plot(list(range(0,175)),list(range(0,175)), color="black", label="One-to-one 1 line")

obs_pred = pd.DataFrame({"pred": [np.exp(x) for x in pred_data["pred"]], "obs":  [np.exp(x) for x in pred_data["obs"]]})
plt.scatter(obs_pred.iloc[:,0], obs_pred.iloc[:,1])
plt.xlabel("SSC (mg/L) Predicted")
plt.ylabel("SSC (mg/L) Observed")
plt.title("Observed vs. Predicted SSC for withheld ITV Test Sites")
plt.legend()

In [ ]:
from src.utils import MultipleRegression
model = MultipleRegression(len(model_out["features"]), len(model_out["layer_out_neurons"]), model_out["layer_out_neurons"], activation_function=eval(f'nn.{model_out["activation"]}'))

with open(f"output/mlp/top_model_tmp.pt", "rb") as f:
    model.load_state_dict(torch.load(f))